In [1]:
import numpy as np
from keras.models import Sequential,Model
from keras.layers import  Dropout,Input,LSTM,Dense,Maximum,ReLU,Conv2D,MaxPool2D,Conv1D,MaxPool1D,Flatten,Concatenate,TimeDistributed,RepeatVector,UpSampling1D,Softmax,Lambda,Reshape,Multiply,Add
from keras import backend as K
from keras import initializers
from tensorflow.keras.utils import plot_model
import tensorflow.keras.callbacks as callbacks
import pandas as pd
import pydot
import pydotplus
from matplotlib import pylab as plt
import datetime
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from unicodedata import name
from requests import session
from imblearn.under_sampling import RandomUnderSampler
import random
import copy
import collections
import math
import pickle

* atusact.csv  
activityに関するデータ  
id，duration，start，stop，場所  

* atuswho.csv  
誰と一緒にactivityを行ったかのデータ  
trwhona:一人で行うactかどうか（0/1）  
tuwho_code:誰と一緒か 18:一人

* atusrost.csv  
terrp:続柄  
teage:年齢  
tesex:性別

* atusresp.csv  
tschenr:修学レベル  
telfs:給与レベル  
TRDTIND1:job code  
TRDTOCC1:job code  
TUDIARYDAY:曜日

In [9]:
df_act = pd.read_csv("./archive/atusact.csv",usecols=["tucaseid","tuactdur24","tustarttim","tustoptime","trtier1p","tewhere"])

In [11]:
df_act.head(10)

,tucaseid,tuactdur24,tustarttim,tustoptime,trtier1p,tewhere
0,20030100013280,60,04:00:00,05:00:00,13,9
1,20030100013280,30,05:00:00,05:30:00,1,-1
2,20030100013280,600,05:30:00,15:30:00,1,-1
3,20030100013280,150,15:30:00,18:00:00,12,1
4,20030100013280,5,18:00:00,18:05:00,11,1
5,20030100013280,175,18:05:00,21:00:00,12,1
6,20030100013280,270,21:00:00,01:30:00,1,-1
7,20030100013280,10,01:30:00,01:40:00,1,-1
8,20030100013280,140,01:40:00,04:30:00,13,9
9,20030100013344,180,04:00:00,07:00:00,1,-1


In [15]:
df_rost = pd.read_csv("./archive/atusrost.csv",usecols=["tucaseid","tulineno","terrp","teage","tesex"])

In [19]:
df_rost[df_rost["tulineno"]==1][["tucaseid","teage","tesex"]].head(10)

,tucaseid,teage,tesex
0,20030100013280,60,1
3,20030100013344,41,2
7,20030100013352,26,2
9,20030100013848,36,2
13,20030100014165,51,1
17,20030100014169,32,2
20,20030100014209,44,2
23,20030100014427,21,2
28,20030100014550,33,2
33,20030100014758,39,2


In [22]:
df_resp = pd.read_csv("./archive/atusresp.csv",usecols=["tucaseid","teschenr","telfs","trdtind1","tudiaryday"])

In [24]:
df_resp["teschenr"].value_counts()

 2    81517
-1    72589
 1    16733
-3        3
Name: teschenr, dtype: int64

In [23]:
df_resp.head(10)

,tucaseid,trdtind1,tudiaryday,teschenr,telfs
0,20030100013280,40,6,-1,2
1,20030100013344,16,7,2,1
2,20030100013352,43,7,2,2
3,20030100013848,-1,5,2,4
4,20030100014165,42,5,-1,1
5,20030100014169,40,5,1,2
6,20030100014209,43,2,2,1
7,20030100014427,41,3,2,1
8,20030100014550,34,7,2,1
9,20030100014758,41,5,2,1


In [2]:
df = pd.read_csv("./US_Data_age_sex.csv")

In [4]:
len(df)

477138

In [8]:
df["tucaseid"][2]

20030100000000.0

In [5]:
df.head(10)

,tucaseid,tulineno,terrp,teage,tesex
0,2.003010e+13,1,18,60,1
1,2.003010e+13,2,20,72,2
2,2.003010e+13,3,22,37,2
3,2.003010e+13,1,18,41,2
4,2.003010e+13,2,20,42,1
5,2.003010e+13,3,22,3,1
6,2.003010e+13,4,22,0,2
7,2.003010e+13,1,18,26,2
8,2.003010e+13,2,20,35,1
9,2.003010e+13,1,18,36,2
